In [1]:
import pandas as pd

df=pd.read_pickle('/kaggle/input/building-hotpotqa-sots-dataset/sots_df.pkl')
df.shape

(11298, 8)

In [2]:
df.head()

,question,context,reason,answer,ground_truth,temperature,tokens,score
0,VIVA Media AG changed it's name in 2004. What ...,"VIVA Media GmbH (until 2004 ""VIVA Media AG"") i...",<think>\nLet’s think through this step by step...,Gesellschaft mit beschränkter Haftung,Gesellschaft mit beschränkter Haftung,0.00,69,0.0
1,VIVA Media AG changed it's name in 2004. What ...,"VIVA Media GmbH (until 2004 ""VIVA Media AG"") i...",<think>\nLet’s think through this step by step...,Gesellschaft mit beschränkter Haftung,Gesellschaft mit beschränkter Haftung,0.25,69,0.0
2,VIVA Media AG changed it's name in 2004. What ...,"VIVA Media GmbH (until 2004 ""VIVA Media AG"") i...",<think>\nLet’s think through this step by step...,Gesellschaft mit beschränkter Haftung,Gesellschaft mit beschränkter Haftung,1.00,69,0.0
3,VIVA Media AG changed it's name in 2004. What ...,"VIVA Media GmbH (until 2004 ""VIVA Media AG"") i...",<think>\nLet’s think through this step by step...,Gesellschaft mit beschränkter Haftung,Gesellschaft mit beschränkter Haftung,1.25,61,0.0
4,VIVA Media AG changed it's name in 2004. What ...,"VIVA Media GmbH (until 2004 ""VIVA Media AG"") i...",<think>\nLet’s think through this step by step...,Gesellschaft mit beschränkter Haftung,Gesellschaft mit beschränkter Haftung,1.50,69,0.0


In [3]:
from doraemon import Doraemon
from relaxed_fda import RelaxedFDA

logger = Doraemon.get_logger(name=__name__, logfile="relaxed_FDA_on_hotpot_qa.log")

grouped = (df.groupby('question', sort=False).agg(reasoning_paths=('reason', list), context=('context', 'first'), ground_truth=('ground_truth', 'first')).reset_index())

2025-06-21 02:17:01.530743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750472221.806216      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750472221.889406      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import torch
from sketch_of_thought import SoT
from sentence_transformers import SentenceTransformer

device=SoT.get_device()

logger.info(device)

encoder_path='/kaggle/input/encoder-l6-v2/transformers/v0.1.4/1'

encoder=SentenceTransformer(encoder_path)
encoder.to(device)

2025-06-21 02:17:20,632 INFO cpu


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [5]:
from typing import List, Dict, Tuple, Optional

# define fewshot here.
DATASET_TYPE=SoT.classify_question(f"Question:{grouped.iloc[0].question}\nContext:{grouped.iloc[0].context}")
logger.info(DATASET_TYPE)

SYSTEM_INSTRUCTION_EN=SoT.get_prompts_en()[DATASET_TYPE]

PROMT=[]

PATTERN=r'\\boxed{([^}]+)}'


PROMT.append({"role": "system", "content": SYSTEM_INSTRUCTION_EN })

# few-shot examples with correct reasoning with wrong answer
# 432
# 411
# 392
# 257
# 232-233
D: List[Dict] = [
    {
        "question": "What company did Rex Maughan acquire?",
        "context": """Forever Living Products International, Inc. (FLPI), founded by Rex Maughan, acquired Aloe Vera of America in the 1990s. FLPI manufactures and sells aloe vera-based drinks and nutritional supplements.""",
        "correct_rs": "<think>\nLet’s think through this step by step\n#Rex_Maughan → acquired Aloe Vera of America\n</think>\n\\boxed{Aloe Vera of America}",
        "wrong_rs": "<think>\nLet’s think through this step by step\n#Rex_Maughan → Forever Living Products International founder\n</think>\n\\boxed{Forever Living Products International}"
    },
    {
        "question": "What building is opposite the ceremonial meeting place of the Accession Council in the United Kingdom?",
        "context": """The Accession Council meets at St James's Palace. Mark Masons' Hall is located at 86 St James's Street, London, opposite St James's Palace.""",
        "correct_rs": "<think>\nLet’s think through this step by step\n#ceremonial_meeting_place → Accession Council meets at St James's Palace\nMark Masons' Hall is located opposite St James's Palace (as per context)\n</think>\n\\boxed{Mark Masons' Hall}",
        "wrong_rs": "<think>\nLet’s think through this step by step\n#ceremonial_meeting_place → St James's Palace\nSt James's Palace is opposite itself\n</think>\n\\boxed{St James's Palace}"
    },
    # {
    #     "question": "What is represented on the cover art of the studio album that includes the song \"Speed of Sound\"?",
    #     "context": """X&Y was released in 2005. The song \"Speed of Sound\" is from Coldplay's third studio album, X&Y, whose cover art is a combination of colours and blocks representing the Baudot code.""",
    #     "correct_rs": "<think>\nLet’s think through this step by step\n#Speed_of_Sound is on Coldplay's third studio album X&Y\nThe cover art for X&Y is a combination of colours and blocks\nThis is a representation of the Baudot code (as per context)\n</think>\n\\boxed{Baudot code}",
    #     "wrong_rs": "<think>\nLet’s think through this step by step\n#album with Speed of Sound → cover_art\nThe cover art shows colours and blocks\n</think>\n\\boxed{Colours and blocks}"
    # },
    # {
    #     "question": "What is the birthday of this Anglo-Irish actress, courtesan, and mistress, who was the mother to the illegitimate daughter of King William IV?",
    #     "context": """She was also known for her stage performances in London. Dorothea Jordan (22 November 1761 – 5 July 1816) was an Anglo-Irish actress and long-time mistress of King William IV, with whom she had several children.""",
    #     "correct_rs": "<think>\nLet’s think through this step by step\n#Dorothea_Jordan → birthday 22 November 1761 (as per context)\n</think>\n\\boxed{22 November 1761}",
    #     "wrong_rs": "<think>\nLet’s think through this step by step\n#Dorothea_Jordan → mother of Elizabeth Hay, Countess of Erroll\n#Elizabeth_Hay → birthday 17 January 1801\n</think>\n\\boxed{17 January 1801}"
    # },
    # {
    #     "question": "Otterington railway station was on a 393 mi major railway that linked Edinburgh to what city?",
    #     "context": """Otterington railway station was on the East Coast Main Line, a 393-mile railway linking Edinburgh and London. The station opened in 1841 and closed in 1958.""",
    #     "correct_rs": "<think>\nLet’s think through this step by step\n#Otterington station is on the East Coast Main Line\nThe East Coast Main Line is a 393 mi railway between London and Edinburgh\nSo, the city linked to Edinburgh is London\n</think>\n\\boxed{London}",
    #     "wrong_rs": "<think>\nLet’s think through this step by step\n#Otterington station → 393 mi railway\n#Edinburgh → Krishna (incorrect line and city)\n</think>\n\\boxed{Krishna}"
    # }
]

OPTIMIZE_SYSTEM_PROMPT = (
    "You are a concise and helpful assistant for conceptual chaining. "
    "Provide step-by-step solutions using minimal tokens, ensuring accuracy. "
    "Conclude with the final answer in the format: \\boxed{answer}. "
    "When given a reasoning process, refine it to be both correct and succinct."
)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

2025-06-21 02:17:28,232 INFO conceptual_chaining


In [6]:
import re
from collections import Counter

def construct_interven_prompt(selected_demos: List[Dict], r_k:str, test_question: str, test_evidence: str)->str:
    prompt = ("")

    for i, demo in enumerate(selected_demos, start=1):
        prompt += (
            f"Demo {i}:\n"
            f"Q: The question is: {demo['question']}\n"
            f"E: The context is: {demo['context']}\n"
            f"The provided reasoning path is: {demo['wrong_rs']}\n"
            f"A: The improved reasoning path is: {demo['correct_rs']}\n"
            f"Therefore, the correct answer is: {RelaxedFDA.get_answer(demo['correct_rs'])}\n\n"
        )
    
    # Append the test question.
    prompt += (
        "Test Example:\n"
        f"Q: The question is: {test_question}\n"
        f"E: The context is : {test_evidence}\n"
        "<think>/nLet us think step by step.\n"
        f"The provided reasoning path is: {r_k}\n"
        "A: The improved reasoning path is: [improved_rs]\n"
        "Therefore, the correct answer is: [answer]\n"
    )

    return prompt

def normalize(text):
    """Normalize text by converting to lowercase and removing non-alphanumeric characters."""
    return re.sub(r'\W+', ' ', text.lower()).strip()

def is_substring_match(a:str, b:str)->bool:
    na=normalize(a)
    nb=normalize(b)
    return na in nb or nb in na

def extract_match(a, b, threshold=0.5):
    """
    Returns True if the overlap of tokens is >= threshold (as a proportion of the shorter string's tokens).
    """
    tokens_a = set(normalize(a).split())
    tokens_b = set(normalize(b).split())
    overlap = tokens_a & tokens_b
    min_len = min(len(tokens_a), len(tokens_b))
    if min_len == 0:
        return False
    return len(overlap) / min_len >= threshold

def enhanced_exact_match(predicted: str, ground_truth: str, token_threshold: float = 0.5) -> bool:
    """
    Returns True if the predicted answer exactly matches, is a substring of, or has sufficient token overlap with the ground truth answer.
    """
    if normalize(predicted) == normalize(ground_truth):
        return True
    if is_substring_match(predicted, ground_truth):
        return True
    if extract_match(predicted, ground_truth, token_threshold):
        return True
    return False


# Precision: The ratio of overlapping tokens to the total number of tokens in the predicted answer.
# Recall: The ratio of overlapping tokens to the total number of tokens in the ground truth answer.
def f1_score_custom(predicted:str, ground_truth:str):
    """Compute F1 score based on token overlap."""
    pred_tokens = normalize(predicted).split()
    gt_tokens = normalize(ground_truth).split()
    common = Counter(pred_tokens) & Counter(gt_tokens)
    num_common = sum(common.values())
    if num_common == 0:
        return 0.0
    precision = num_common / len(pred_tokens)
    recall = num_common / len(gt_tokens)
    return 2 * (precision * recall) / (precision + recall)

In [7]:
correct_em = 0
total = 0
f1_total = 0.0

for _, row in grouped.iterrows():
    q = row['question']
    r_p = row['reasoning_paths']
    context=row['context']

    final_answer, o_p = RelaxedFDA.causal_infer_with_fdr(
        q,
        context,
        logger,
        OPTIMIZE_SYSTEM_PROMPT,
        encoder,
        r_p,
        D,
        enable_logger_rs=False
    )
    logger.info(f"The final answer: {final_answer}")

    ground_truth = row['ground_truth']

    if final_answer is not None and ground_truth is not None:
        final_answer_clean = RelaxedFDA.get_answer(final_answer)
        logger.info(f"Question is {q}")
        logger.info(f"Final answer is {str(final_answer_clean)}, ground truth is {ground_truth}")

        norm_pred=normalize(final_answer_clean)
        norm_truth=normalize(ground_truth)

        if enhanced_exact_match(final_answer_clean, ground_truth):
            correct_em += 1
        else:
            logger.warning(f"Mismatch: predicted={final_answer_clean}, expected={ground_truth}")

        f1=f1_score_custom(final_answer_clean, ground_truth)
        f1_total += f1

        total += 1
    else:
        logger.warning(f"Missing data: predicted={final_answer}, expected={ground_truth}")

2025-06-21 02:17:29,009 INFO ====================================================================================================
2025-06-21 02:17:29,010 INFO Generated 7 valid reasoning path and embeddings.
2025-06-21 02:17:29,083 INFO Cluster 1: |C_0| = 2, P(r_0|do(X)) ≈ 0.29
2025-06-21 02:17:32,955 INFO Cluster 2: |C_1| = 3, P(r_1|do(X)) ≈ 0.43
2025-06-21 02:17:35,845 INFO Cluster 3: |C_2| = 1, P(r_2|do(X)) ≈ 0.14
2025-06-21 02:17:38,472 INFO Cluster 4: |C_3| = 1, P(r_3|do(X)) ≈ 0.14
2025-06-21 02:17:41,082 INFO Aggregated candidate votes (weighted): {'Gesellschaft mit beschränkter Haftung': 0.7142857142857142, 'Gesellschaft\\ mit\\ beschränkter\\ Haftung': 0.23809523809523808}
2025-06-21 02:17:41,083 INFO Final aggregated answer is Gesellschaft mit beschränkter Haftung and weight 0.7142857142857142
2025-06-21 02:17:41,084 INFO Overall estimated probability (aggregated): 0.75
2025-06-21 02:17:41,086 INFO The final answer: Gesellschaft mit beschränkter Haftung
2025-06-21 02:17:41,087

In [8]:
# After processing all rows
em_score = correct_em / total if total > 0 else 0.0
avg_f1 = f1_total / total if total > 0 else 0.0
logger.info(f"Exact Match (EM): {em_score:.2%}")
logger.info(f"Average F1 Score: {avg_f1:.2%}")

2025-06-21 07:07:44,675 INFO Exact Match (EM): 84.92%
2025-06-21 07:07:44,676 INFO Average F1 Score: 75.14%
